In [ ]:
# multivariate multi-step stacked lstm example
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import pickle as pkl
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score

In [ ]:
nthu_train_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_X", 'rb'))
nthu_train_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_y", 'rb'))

nthu_test_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_X", 'rb'))
nthu_test_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_y", 'rb'))

In [ ]:
nthu_train_y=np.array([int(nthu_train_y[i][0]/2) for i in range(len(nthu_train_y))])
nthu_test_y=np.array([int(nthu_test_y[i][0]/2) for i in range(len(nthu_test_y))])

In [ ]:
n_steps_in, n_features, n_steps_out= 150, 8, 1

# define model
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(n_steps_out, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
# fit model
model.fit(nthu_train_X, np.ravel(nthu_train_y), batch_size=8, epochs=4, verbose=2)

In [ ]:
# demonstrate prediction
yhat = model.predict_classes(nthu_test_X, verbose=1)
print(yhat)

In [ ]:
mean_squared_error(np.squeeze(yhat),nthu_test_y)

In [ ]:
def display_test_scores(test, pred):
    str_out = ""
    str_out += ("TEST SCORES\n")
    str_out += ("\n")

    #print accuracy
    accuracy = accuracy_score(test, pred)
    str_out += ("ACCURACY: {:.4f}\n".format(accuracy))
    str_out += ("\n")

    #print AUC score
    auc = roc_auc_score(test, pred)
    str_out += ("AUC: {:.4f}\n".format(auc))
    str_out += ("\n")

    #print confusion matrix
    str_out += ("CONFUSION MATRIX:\n")
    conf_mat = confusion_matrix(test, pred)
    str_out += ("{}".format(conf_mat))
    str_out += ("\n")
    str_out += ("\n")

    #print FP, FN
    str_out += ("FALSE POSITIVES:\n")
    fp = conf_mat[1][0]
    pos_labels = conf_mat[1][0]+conf_mat[1][1]
    str_out += ("{} out of {} positive labels ({:.4f}%)\n".format(fp, pos_labels,fp/pos_labels))
    str_out += ("\n")

    str_out += ("FALSE NEGATIVES:\n")
    fn = conf_mat[0][1]
    neg_labels = conf_mat[0][1]+conf_mat[0][0]
    str_out += ("{} out of {} negative labels ({:.4f}%)\n".format(fn, neg_labels, fn/neg_labels))
    str_out += ("\n")

    #print classification report
    str_out += ("PRECISION, RECALL, F1 scores:\n")
    str_out += ("{}".format(classification_report(test, pred)))
    
    false_indexes = np.where(test != pred)
    return str_out, false_indexes

In [ ]:
y_pred = np.squeeze(yhat)
results, false = display_test_scores(nthu_test_y, y_pred)
print(results)
y_pred.shape

In [ ]:
model.save("lstm_stacked_v2.h5")